In [16]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from scipy.stats import skew, kurtosis, normaltest
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error,roc_auc_score,precision_score, recall_score,accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer

In [2]:
!wget -O ./data/car_fuel_efficiency.csv wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv

Prepended http:// to 'wget'
--2025-11-02 21:12:22--  http://wget/
Resolving wget (wget)... failed: nodename nor servname provided, or not known.
wget: unable to resolve host address ‘wget’
--2025-11-02 21:12:22--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/car_fuel_efficiency.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 874188 (854K) [text/plain]
Saving to: ‘./data/car_fuel_efficiency.csv’

./data/car_fuel_eff 100%[===================>] 853.70K  --.-KB/s    in 0.02s   

2025-11-02 21:12:23 (45.8 MB/s) - ‘./data/car_fuel_efficiency.csv’ saved [874188/874188]

FINISHED --2025-11-02 21:12:23--
Total wall clock time: 0.2s
Downloaded: 1 files, 854K in 0.02s (45.8 MB/s)


In [3]:
df = pd.read_csv("./data/car_fuel_efficiency.csv")

In [4]:
df_filled

NameError: name 'df_filled' is not defined

In [ ]:
print("Original:")
for column in df.columns:
    print(f"{column} missing {df[column].isnull().sum()} records")

df_filled = df.copy()

df_filled['num_cylinders'] = df['num_cylinders'].fillna(0)
df_filled['horsepower'] = df['horsepower'].fillna(0)
df_filled['num_doors'] = df['num_doors'].fillna(0)
df_filled['acceleration'] = df['acceleration'].fillna(0)

print("Filled:")
for column in df.columns:
    print(f"{column} missing {df_filled[column].isnull().sum()} records")

Original:
engine_displacement missing 0 records
num_cylinders missing 482 records
horsepower missing 708 records
vehicle_weight missing 0 records
acceleration missing 930 records
model_year missing 0 records
origin missing 0 records
fuel_type missing 0 records
drivetrain missing 0 records
num_doors missing 502 records
fuel_efficiency_mpg missing 0 records
Filled:
engine_displacement missing 0 records
num_cylinders missing 0 records
horsepower missing 0 records
vehicle_weight missing 0 records
acceleration missing 0 records
model_year missing 0 records
origin missing 0 records
fuel_type missing 0 records
drivetrain missing 0 records
num_doors missing 0 records
fuel_efficiency_mpg missing 0 records


In [5]:
y = df['fuel_efficiency_mpg']
X = df.drop(columns=['fuel_efficiency_mpg'])


X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.4, random_state=1
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=1
)


print(f"df_train: {len(X_train)} rows")
print(f"df_val:   {len(X_val)} rows")
print(f"df_test:  {len(X_test)} rows")

df_train: 5822 rows
df_val:   1941 rows
df_test:  1941 rows


In [6]:
X_train.head()

,engine_displacement,num_cylinders,horsepower,vehicle_weight,acceleration,model_year,origin,fuel_type,drivetrain,num_doors
4150,150,5.0,NaN,2510.410896,10.0,2010,Europe,Gasoline,Front-wheel drive,0.0
5660,200,3.0,181.0,3597.483663,12.7,2006,Europe,Diesel,Front-wheel drive,0.0
7319,220,6.0,118.0,2217.349625,NaN,2004,USA,Diesel,All-wheel drive,-2.0
1562,110,3.0,136.0,3307.407159,15.9,2010,USA,Gasoline,All-wheel drive,-2.0
7499,220,2.0,190.0,3315.299773,18.3,2017,USA,Diesel,Front-wheel drive,1.0


In [7]:
y = df["fuel_efficiency_mpg"].values
X_train_dicts = X_train.to_dict(orient="records")
X_val_dicts   = X_val.to_dict(orient="records")
X_test_dicts  = X_test.to_dict(orient="records")

dv = DictVectorizer(sparse=False)
X_train_enc = dv.fit_transform(X_train_dicts)
X_val_enc   = dv.transform(X_val_dicts)
X_test_enc  = dv.transform(X_test_dicts)


In [14]:
dv.get_feature_names_out()

array(['acceleration', 'drivetrain=All-wheel drive',
       'drivetrain=Front-wheel drive', 'engine_displacement',
       'fuel_type=Diesel', 'fuel_type=Gasoline', 'horsepower',
       'model_year', 'num_cylinders', 'num_doors', 'origin=Asia',
       'origin=Europe', 'origin=USA', 'vehicle_weight'], dtype=object)

In [20]:
y_pred

array([16.85731164, 12.8722818 , 12.8722818 , ..., 12.8722818 ,
       12.8722818 , 12.8722818 ], shape=(1941,))

### Question 1

In [ ]:
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train_enc, y_train)

y_pred = dt.predict(X_val_enc)
print(mean_squared_error(y_val, y_pred))

2.5399474400882105


In [ ]:
from sklearn.tree import export_text
print(export_text(dt, feature_names=dv.get_feature_names_out()))


|--- vehicle_weight <= 3028.82
|   |--- value: [16.86]
|--- vehicle_weight >  3028.82
|   |--- value: [12.87]



### Question 2

In [10]:
y

array([13.23172891, 13.68821744, 14.246341  , ..., 17.18658678,
       15.33155059, 14.8844674 ], shape=(9704,))

In [9]:
X_train_enc

array([[1.00000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 2.51041090e+03],
       [1.27000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 3.59748366e+03],
       [           nan, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 2.21734962e+03],
       ...,
       [1.65000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        1.00000000e+00, 0.00000000e+00, 2.59665719e+03],
       [1.22000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.86038802e+03],
       [1.46000000e+01, 0.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 3.40135769e+03]], shape=(5822, 14))